In [12]:
from jikanpy import Jikan
import json 
import pandas as pd 
import numpy as np
from time import sleep

In [13]:
jikan = Jikan()

In [20]:
archive = jikan.season_archive()

In [95]:
# example of request for a specific season of a specific year 
winter_2018_anime = jikan.season(year=2018, season='fall')

In [98]:
for anime in winter_2018_anime['anime']:
    anime['year'] = "2018"
    anime['season'] = 'fall'

In [14]:
# get anime for a single year 
seasons = ['winter', 'spring', 'summer', 'fall']
def get_anime_year(year):
    anime_year = []
    for season in seasons: 
        # get all the anime for that season 
        try: 
            anime_season = jikan.season(year=year, season=season)['anime']
            for anime in anime_season: 
                anime['air_year'] = year 
                anime['air_season'] = season
            anime_year.extend(anime_season)
        except: 
            continue
    return anime_year

In [26]:
# get all anime since 1920 
all_anime = []

In [27]:
for year in range(1917, 2022):
    anime_year = get_anime_year(year)
    sleep(5)
    all_anime.extend(anime_year)

In [28]:
len(all_anime)

22474

In [29]:
with open('mal_scrape.json', 'w') as outfile:
    json.dump(all_anime, outfile)

In [30]:
with open('mal_scrape.json') as json_file:
    data = json.load(json_file)

# Data formatting

In [31]:
df = pd.DataFrame.from_dict(data)

In [32]:
df.columns

Index(['mal_id', 'url', 'title', 'image_url', 'synopsis', 'type',
       'airing_start', 'episodes', 'members', 'genres', 'source', 'producers',
       'score', 'licensors', 'r18', 'kids', 'continuing', 'air_year',
       'air_season'],
      dtype='object')

In [33]:
# get list of genres 
df['genres_clean'] = df['genres'].apply(lambda x : [element['name'] for element in x])
# get list of producers 
df['producers_clean'] = df['producers'].apply(lambda x : [element['name'] for element in x])
# drop the anime for which there is no start date, note that there is only about 100 of them 
df = df.dropna(subset=['airing_start'])

In [34]:
# get the dates in datetime format 
df['airing_start_clean'] = df['airing_start'].apply(lambda x : x.split("T")[0])
df['airing_start_clean'] = pd.to_datetime(df['airing_start_clean'])
df['year'] = df['airing_start_clean'].dt.year

In [35]:
df.head()

mal_id                                                url  \
1    6654  https://myanimelist.net/anime/6654/Namakura_Ga...   
2   10742  https://myanimelist.net/anime/10742/Saru_to_Ka...   
6   23187  https://myanimelist.net/anime/23187/Chamebou_S...   
7   18457  https://myanimelist.net/anime/18457/Hanasaka_J...   
8   23183   https://myanimelist.net/anime/23183/Itazura_Post   

                                               title  \
1                                    Namakura Gatana   
2                             Saru to Kani no Gassen   
6  Chamebou Shin Gachou: Nomi Fuufu Shikaeshi no ...   
7                                     Hanasaka Jijii   
8                                       Itazura Post   

                                           image_url  \
1  https://cdn.myanimelist.net/images/anime/4/152...   
2  https://cdn.myanimelist.net/images/anime/4/837...   
6       https://cdn.myanimelist.net/images/qm_50.gif   
7  https://cdn.myanimelist.net/images/anime/5/494...   
8       https://cdn.myanimelist.net/images/qm_50.gif   

                                            synopsis   type  \
1  Namakura Gatana, meaning "dull-edged sword," i...  Movie   
2  A monkey tricks a crab and steals his food. Mo...  Movie   
6                                 (No synopsis yet.)  Movie   
7  Short movie produced by the first generation a...  Movie   
8                                 (No synopsis yet.)  Movie   

                airing_start  episodes  members  \
1  1917-06-29T15:00:00+00:00       1.0     5505   
2  1917-05-19T15:00:00+00:00       1.0      604   
6  1917-04-27T15:00:00+00:00       1.0      276   
7  1917-08-25T15:00:00+00:00       1.0      760   
8  1917-07-27T15:00:00+00:00       1.0      282   

                                              genres  ... licensors    r18  \
1  [{'mal_id': 4, 'type': 'anime', 'name': 'Comed...  ...        []  False   
2  [{'mal_id': 8, 'type': 'anime', 'name': 'Drama...  ...        []  False   
6  [{'mal_id': 4, 'type': 'anime', 'name': 'Comed...  ...        []  False   
7  [{'mal_id': 4, 'type': 'anime', 'name': 'Comed...  ...        []  False   
8  [{'mal_id': 4, 'type': 'anime', 'name': 'Comed...  ...        []  False   

    kids continuing  air_year  air_season       genres_clean  producers_clean  \
1  False      False      1917      spring  [Comedy, Samurai]               []   
2  False      False      1917      spring            [Drama]               []   
6  False      False      1917      spring           [Comedy]               []   
7  False      False      1917      summer    [Comedy, Drama]               []   
8  False      False      1917      summer           [Comedy]               []   

  airing_start_clean  year  
1         1917-06-29  1917  
2         1917-05-19  1917  
6         1917-04-27  1917  
7         1917-08-25  1917  
8         1917-07-27  1917  

[5 rows x 23 columns]

In [36]:
df.to_csv("mal_scrape.csv")